In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed


In [12]:
Z_eff = pd.read_csv('drive/MyDrive/ML Project/data_Z_eff.csv',header=None)
R_eff = pd.read_csv('drive/MyDrive/ML Project/data_R_eff.csv',header = None)
R_org_eff = pd.read_csv('drive/MyDrive/ML Project/data_R_org_eff.csv',header = None)
group_ind = pd.read_csv('drive/MyDrive/ML Project/data_group_ind.csv',header =None)

In [13]:
Z_eff.shape

(2219316, 64)

In [14]:

t_train_start = list(range(0,41*12+1,12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x+12 for x in t_test_start]
t_test_end[41]=678

In [15]:
train_start = group_ind.iloc[t_train_start][0].to_list()
train_end = group_ind.iloc[t_train_end][0].to_list()
test_start = group_ind.iloc[t_test_start][0].to_list()
test_end = group_ind.iloc[t_test_end][0].to_list()
val_start = group_ind.iloc[t_val_start][0].to_list()
val_end = group_ind.iloc[t_val_end][0].to_list()                         

In [16]:
def portf_ret(preds, y_org):
    list_buy = [idx for idx, val in preds if val > np.percentile(preds,90)]
    list_buy = [idx for idx, val in preds if val < np.percentile(preds,10)]
    portf_ret_t = np.mean(y_true_t[list_buy]) - np.mean(y_true_t[list_sell])
    return portf_ret_t

In [17]:
y_pred = []
y_pred_val = []
y_pred_test = []
loss = []
model_list = []
y_org = []
y_org_val = []
y_org_test = []
y_reff = []
y_reff_val = []
y_reff_test = []
portf_ret_nn_test = []
R_pred=R_org_eff
seed(786)
tf.random.set_seed(786)
seed(1)

In [18]:
for retrain_idx in range(42):
    X = Z_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Z_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    X_test = Z_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    X_val = X_val.loc[:,X_idx]
    X_test = X_test.loc[:,X_idx]
    y =R_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    y_val = R_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    y_test = R_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    yorg = R_org_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    yorg_val = R_org_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    yorg_test=R_org_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(256,activation='LeakyReLU'))
    model.add(BatchNormalization())
    model.add(Dense(128,activation='LeakyReLU'))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='LeakyReLU'))
    model.add(BatchNormalization())
    model.add(Dense(32 ,activation='LeakyReLU'))
    model.add(BatchNormalization())
    model.add(Dense(16,activation='LeakyReLU'))
    model.add(Dense(1,activation='LeakyReLU'))
    Optimizer_func = tf.keras.optimizers.SGD()
    model.compile(optimizer=Optimizer_func,loss='mse')
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    seed(786)
    tf.random.set_seed(786)
    seed(1)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=512, epochs=50,verbose=1,callbacks=[early_stop])
    preds = model.predict(X_test).flatten()
    y_org_test = yorg_test[0].to_list()
    R_pred.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]=pd.DataFrame(preds)
    for t in range(t_test_start[retrain_idx], t_test_end[retrain_idx]):
        month_start = group_ind.iloc[t][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        month_end = group_ind.iloc[t+1][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        preds_month = preds[month_start:month_end]
        yorg_month = y_org_test[month_start:month_end]
        list_buy = [i for i, val in enumerate(preds_month) if val > np.quantile(preds_month,.9)]
        list_sell = [i for i, val in enumerate(preds_month) if val < np.quantile(preds_month,.1)]
        print(t,month_start,month_end,len(preds_month),len(list_buy),len(list_sell))
        portf_ret_nn_test.append(np.mean([yorg_month[i] for i in list_buy]) - np.mean([yorg_month[i] for i in list_sell]))
    loss.append(min(history.history['val_loss']))
    model_list.append(model)

Epoch 1/50
412/412 [==============================] - 8s 17ms/step - loss: 0.0512 - val_loss: 0.0161
Epoch 2/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0111 - val_loss: 0.0116
Epoch 3/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0095 - val_loss: 0.0106
Epoch 4/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0090 - val_loss: 0.0101
Epoch 5/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0088 - val_loss: 0.0100
Epoch 6/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0087 - val_loss: 0.0100
Epoch 7/50
412/412 [==============================] - 6s 16ms/step - loss: 0.0086 - val_loss: 0.0100
Epoch 8/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0085 - val_loss: 0.0102
Epoch 9/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0085 - val_loss: 0.0102
Epoch 10/50
412/412 [==============================] - 7s 16ms/step - loss: 0.0084 - val_lo

In [20]:
np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12)

7.24551797860896
